In [12]:
# default_exp fontlearner
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Font Learner

> Diffvg-based learner for font optimisation

In [2]:
#hide
from nbdev.showdoc import *

In [29]:
#export
from collections.abc import Callable
import glob
import IPython.display
import numpy as np
import os
import pydiffvg
import subprocess
import torch
from torch import Tensor
from typing import Union

class Scene:
    """Just a utility to hold the different scene arguments together.
       The folder argument is optional."""
    def __init__(self, shapes: list[any], shape_groups: list[pydiffvg.ShapeGroup], 
        canvas_width = 256, canvas_height = 256, iter_name = "iter_", folder: str = None):

        assert shapes is not None and len(shapes) != 0
        assert shape_groups is not None and len(shape_groups) != 0

        self.shapes = shapes
        self.shape_groups = shape_groups
        self.canvas_width = canvas_width
        self.canvas_height = canvas_height
        self.iter_name = iter_name
        self.folder = folder
        self.grad_pattern = "grad_"
        self._iteration = -1
        self._seed = -1

    def get_folder(self, folder: str = None) -> str:
        """A shorthand for parsing the folder argument."""
        folder = self.folder if folder is None else folder
        assert folder is not None
        return folder[:-1] if folder.endswith("/") else folder

    def get_scene_args(self) -> list:
        return pydiffvg.RenderFunction.serialize_scene(self.canvas_width, 
                                                       self.canvas_height, 
                                                       self.shapes, 
                                                       self.shape_groups)
    
    def new_seed(self) -> int:
        """For use with render"""
        self._seed += 1
        return self._seed

    def next_i(self) -> int:
        """For use with render"""
        self._iteration += 1
        return self._iteration

    def normalise(self, values: Union[list, float]) -> Tensor:
        """Create a normalised, grad Tensor from the possible multidim array of points"""
        return torch.tensor(np.array(values) / self.canvas_width, 
                            dtype=torch.float32, 
                            requires_grad=True)

    def denormalise(self, normalised: Tensor) -> Tensor:
        """Expand a normalised Tensor"""
        return normalised * self.canvas_width

    def render(self, name: str = None, folder: str = None, render_grad = False) -> Tensor:
        """Render the scene, save as an image and return the image Tensor."""

        i = self.next_i()
        name = f"{self.iter_name}{i}" if name is None else name
        assert name is not None

        folder = self.get_folder(folder)
        scene_args = self.get_scene_args()
        w = self.canvas_width
        h = self.canvas_height
        s = self.new_seed()

        img = pydiffvg.RenderFunction.apply(w, h, 2, 2, s, None, *scene_args)
        # The output image is in linear RGB space. Do Gamma correction before saving the image.
        pydiffvg.imwrite(img.cpu(), f"{folder}/{name}.png", gamma=2.2)

        if render_grad:
            grad = pydiffvg.RenderFunction.render_grad(torch.ones(w, h, 4, device=pydiffvg.get_device()),
                                                    w, h, 2, 2, s, None, *scene_args)
            pydiffvg.imwrite(grad[:,:,0].cpu(), f"{folder}/{self.grad_pattern}{i}.png", gamma=2.2)

        return img

    def render_result_video(self, folder: str = None, iter_pattern: str = None, 
        out_file="0_out", render_grad = False, delete_imgs = False) -> None:
        """Render intermediate images and optionally grad as a video."""
        
        folder = self.get_folder(folder)
        if iter_pattern is None:
            iter_pattern = self.iter_name

        def _run(pat = iter_pattern, out = out_file): 
            subprocess.run(["ffmpeg", 
                            "-f", "lavfi", 
                            "-i", f"color=c=white:s={self.canvas_width}x{self.canvas_height}:r=24",
                            "-framerate", "24", 
                            "-i", f"{folder}/{pat}%d.png", 
                            "-filter_complex", "[0:v][1:v]overlay=shortest=1,format=yuv420p[out]",
                            "-y",
                            "-map", "[out]",
                            f"{folder}/{out}.mp4"],
                            capture_output=True)

        def _rm(pat = iter_pattern):
            # Can't use subprocess with globs
            files = glob.glob(f"{folder}/{pat}*.png")
            for f in files:
                os.remove(f)

        _run()
        if render_grad: 
            _run(self.grad_pattern, f"{self.grad_pattern}out")

        if delete_imgs:
            _rm()
            if render_grad: 
                _rm(self.grad_pattern)

        print("Rendering video done!")
    

In [ ]:
show_doc(Scene.normalise)
show_doc(Scene.denormalise)
show_doc(Scene.render)
show_doc(Scene.render_result_video)

In [9]:
#export
class VectorLearner:
    """Handle optimisation and iterations.
       TODO: Convert to a subclass of Learner."""
    def __init__(self, parameters: list[Tensor], forward: Callable[..., Tensor], 
        target: Tensor = None, loss: Callable[..., Tensor] = None, 
        lr = 1e-2):
        self._lr = lr
        self.parameters = parameters
        self.optimizer = torch.optim.Adam(self.parameters, lr=self._lr)
        self.forward = forward
        self.loss = self.loss_l2 if loss is None else loss
        self.target = target
        self.debug = False
        self.iterations = -1
        self._iteration = -1

    @property
    def lr(self) -> float:
        return self._lr

    @lr.setter
    def lr(self, val: float):
        self._lr = self.optimizer.lr = val

    def loss_l2(self, img: Tensor, target: Tensor, **kwargs) -> Tensor:
        """L2 loss function"""
        return (img - target).pow(2).sum() # mean()

    def next_i(self) -> int:
        """For use with step"""
        self._iteration += 1
        return self._iteration

    def reset_i(self) -> None:
        self._iteration = -1

    def run(self, iterations=100, **kwargs) -> None:
        """Perform iterations number of steps. Kwargs are passed to step and hence to forward."""
        self.reset_i()
        self.iterations = iterations
        for _ in range(iterations):
            self.step(**kwargs)

    def step(self, target: Tensor = None, **kwargs) -> None:
        """Perform one optimisation step. Kwargs are passed to forward."""

        if target is None: target = self.target
        assert target is not None

        self.optimizer.zero_grad()
        # Forward pass: render the image.
        img = self.forward(**kwargs)
        loss = self.loss(img=img, target=target, parameters=self.parameters)
        # Backpropagate the gradients.
        loss.backward()
        if self.debug: 
            for p in self.parameters: print(p.grad)
        # Take a gradient descent step.
        self.optimizer.step()

        print(f"Iteration {self.next_i()}/{self.iterations} • Loss: {loss.item():8.0f}", end="\r")

In [31]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_fontsampler.ipynb.
Converted 02_ocrlearner.ipynb.
Converted 03_fontlearner.ipynb.
Converted index.ipynb.
